# Get Started — reading and validating a segmented model

This notebook demonstrates a small, practical workflow for reading a segmented 3D model from an external raw file, performing quick structure and quality checks, visualizing slices, and exporting the model. The focus is on short, reproducible checks you should run when importing any new dataset.

## What we'll do
- Import a segmented volume from disk using `drp_template.io.import_model`.
- Check basic properties: shape, dtype, min/max values, unique labels and their counts.
- Run a simple labeling helper for binary data (or adapt for multi-class).
- Compute phase fractions and display orthographic slices.
- Export the processed model with metadata.

Notes: `drp_template.io` contains import/export convenience functions used for reading/writing models. `label_binary` is a small helper for binary data; for multi-class segmentation, use the suitable tools under `drp_template.tools`. `phase_fractions` provides a volumetric summary and `ortho_views` makes quick visualization slices.

In [ ]:
# Import the helpers used in this example
import drp_template.io as io
import drp_template.image as drp_image
import numpy as np

# small utilities from the package used below
from drp_template.tools import label_binary
from drp_template.compute import phase_fractions

In [ ]:
# Provide paths to your segmented data and params file.
# Update these paths if your files are located elsewhere.
data_segmented = '../data_segmented_uint8_sandstone_400cube.raw'
paramsfile_input = 'sandstone_400cube.json'

In [ ]:
# Import the segmented model from disk. Make sure dtype and dimensions match the file.
# Sometime the order of dimensions may need to be adjusted depending on how the raw file was saved.
# i.e. some files may be saved in z,x,y order instead of x,y,z.

data = io.import_model(
    data_segmented,
    dtype='uint8',
    dimensions={'nz':400, 'nx':400, 'ny':400},
    voxel_size=4.0,
    paramsfile=paramsfile_input,
)

In [ ]:
# Quick quality checks after import
print('ndim:', getattr(data, 'ndim', None))
print('shape:', getattr(data, 'shape', None))
print('dtype:', getattr(data, 'dtype', None))

### Labeling and phase mapping

The next step converts the raw segmentation into a `labels` object used by downstream tools. The `paramsfile` often contains mappings between integer codes and human-readable phase names. For this sandstone, the segemented phases are the following: Pore, Quartz, Feldspar, Calcite, Pyrite, Quartz-Grainboundary.
You can use label_binary for an active process of labeling segmented data, or you can simply provide a dict of labels.

In [ ]:
# Create labels suitable for analysis and plotting
labels_dict = label_binary(data=data, paramsfile=paramsfile_input)

In [ ]:
# Provide a dictionary of phase labels
from drp_template.default_params import update_parameters_file

labels_dict = {
    0: 'Pore',
    1: 'Quartz',
    2: 'Feldspar',
    3: 'Calcite',
    4: 'Pyrite',
    5: 'Quartz-Grainboundary',
}

update_parameters_file(paramsfile=paramsfile_input, labels=labels_dict)

In [ ]:
# Compute volumetric phase fractions.
fraction = phase_fractions(data=data, labels=labels_dict, paramsfile=paramsfile_input)
print(fraction)

### Quick visualization — orthogonal slices

Choose a few slice indices through the volume to inspect cross-sections. If you provide `voxel_size` the axes will be scaled to real units; otherwise indices are plotted.

In [ ]:
fig, ax = drp_image.ortho_views(data, voxel_size=None, title='Segmented Model', labels=labels_dict, slice_indices=[100, 200, 300])

drp_image.save_figure(fig, 'segmented_model_ortho_views.png')


### Exporting the model

When you're happy with the checks and visual inspection, export the model using `io.export_model`. The function writes the data and associated metadata so it can be re-used or shared. Check the `order` parameter if your downstream tools expect a specific memory layout. If you set 'ensure_unique_params' to False you will overwrite the existing paramsfile with the name you provide. Otherwise uniqueness is handled to avoid overwrites.

In [ ]:
# Export the model and parameters to disk.
io.export_model(filename='sandstone_400cube', data=data, voxel_size=4.0, labels=labels_dict, paramsfile=paramsfile_input, ensure_unique_params=False, order='F')